In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/ast-finetuned-pd.zip", "r") as zip_ref:
    zip_ref.extractall("ast-finetuned-pd")
print("✅ Giải nén thành công!")


✅ Giải nén thành công!


In [ ]:
!ls -lh ast-finetuned-pd


total 329M
-rw-r--r-- 1 root root  650 Nov 10 23:06 config.json
-rw-r--r-- 1 root root 329M Nov 10 23:06 model.safetensors
-rw-r--r-- 1 root root  297 Nov 10 23:06 preprocessor_config.json
-rw-r--r-- 1 root root 5.8K Nov 10 23:06 training_args.bin


In [ ]:
# !pip uninstall pyngrok -y
!pip install pyngrok


In [ ]:

pip install streamlit -q

In [ ]:
pip install streamlit mne torch torchvision torchaudio transformers captum matplotlib numpy


In [ ]:
%%writefile app.py

import streamlit as st
import os
import tempfile
import numpy as np
import mne
import torch
import torch.nn as nn
from transformers import ASTForAudioClassification, AutoFeatureExtractor
from captum.attr import IntegratedGradients

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as mpatches

# ==================== CẤU HÌNH TRANG WEB ====================
st.set_page_config(
    page_title="Chẩn đoán Parkinson từ EEG",
    page_icon="🧠",
    layout="wide"
)

# Các hàm load_models, xử lý AI (process_bdf_for_ast, predict_with_ast, etc.) giữ nguyên như cũ
# ... (Giả sử các hàm từ phần trước được dán vào đây)
# ==================== TẢI MÔ HÌNH ====================
@st.cache_resource
def load_models():
    device = torch.device("cpu")
    MODEL_PATH_AST = 'thang(AST_model)/ast-finetuned-pd'
    MODEL_PATH_CNN = 'lenh(xAI)/eeg_cnn_model_weights.pth'
    model_ast, feature_extractor_ast, model_cnn = None, None, None
    try:
        feature_extractor_ast = AutoFeatureExtractor.from_pretrained(MODEL_PATH_AST)
        model_ast = ASTForAudioClassification.from_pretrained(MODEL_PATH_AST, ignore_mismatched_sizes=True).to(device)
        model_ast.eval()
    except Exception as e:
        st.error(f"Lỗi tải mô hình AST: {e}")
    class EEGCNN1D(nn.Module):
        def __init__(self, n_channels=40, n_classes=2):
            super().__init__()
            self.conv1 = nn.Conv1d(n_channels, 64, kernel_size=5, padding=2); self.bn1 = nn.BatchNorm1d(64)
            self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2); self.bn2 = nn.BatchNorm1d(128)
            self.pool = nn.AdaptiveAvgPool1d(1); self.fc = nn.Linear(128, n_classes); self.relu = nn.ReLU()
        def forward(self, x):
            x = self.relu(self.bn1(self.conv1(x))); x = self.relu(self.bn2(self.conv2(x)))
            x = self.pool(x).squeeze(-1); x = self.fc(x); return x
    try:
        model_cnn = EEGCNN1D(n_channels=40, n_classes=2)
        model_cnn.load_state_dict(torch.load(MODEL_PATH_CNN, map_location=device))
        model_cnn.to(device); model_cnn.eval()
    except FileNotFoundError:
        st.warning(f"Không tìm thấy file '{MODEL_PATH_CNN}'. Chức năng XAI sẽ bị vô hiệu hóa.")
    except Exception as e:
        st.error(f"Lỗi tải mô hình CNN: {e}"); model_cnn = None
    return model_ast, feature_extractor_ast, model_cnn, device
# ==================== CÁC HÀM XỬ LÝ CHO AI ====================
@st.cache_data # Cache lại kết quả xử lý nặng
def run_full_analysis(file_content):
    with tempfile.NamedTemporaryFile(delete=False, suffix='.bdf') as tmp_file:
        tmp_file.write(file_content)
        tmp_file_path = tmp_file.name

    # Giả lập st.status vì không thể dùng trực tiếp trong hàm cache
    print("Bắt đầu phân tích...")

    # 1. AST Analysis
    epochs_ast = process_bdf_for_ast(tmp_file_path)
    preds_ast, probs_ast = predict_with_ast(epochs_ast, model_ast, feature_extractor_ast, device)
    analysis_result = analyze_ast_results(preds_ast, probs_ast)
    epoch_chart_fig = plot_epoch_predictions(probs_ast, preds_ast)

    # 2. CNN Explanation
    explanation_fig = None
    if model_cnn:
        explanation_fig = generate_cnn_explanation_image(tmp_file_path, model_cnn, device)

    # 3. Visualization Data
    eeg_data_viz, sfreq_viz, ch_names_viz = load_eeg_bdf_for_viz(tmp_file_path)
    band_powers_viz, freqs_viz, psd_viz = compute_band_powers(eeg_data_viz, sfreq_viz)

    os.remove(tmp_file_path)
    print("Phân tích hoàn tất!")

    return (analysis_result, epoch_chart_fig, explanation_fig,
            eeg_data_viz, sfreq_viz, ch_names_viz,
            band_powers_viz, freqs_viz, psd_viz)

# (Loại bỏ st.status khỏi các hàm con để tương thích với @st.cache_data)
def process_bdf_for_ast(file_path):
    raw=mne.io.read_raw_bdf(file_path,preload=True,verbose=False);raw.resample(16000);raw.filter(0.5,50,verbose=False)
    ch_name="Cz" if "Cz" in raw.ch_names else raw.ch_names[0];signal_data=raw.get_data()[raw.ch_names.index(ch_name)]
    epoch_len=int(2*16000);epochs=[signal_data[i*epoch_len:(i+1)*epoch_len]/(np.max(np.abs(signal_data[i*epoch_len:(i+1)*epoch_len]))+1e-9) for i in range(len(signal_data)//epoch_len)]
    return [e.astype(np.float32) for e in epochs]
def predict_with_ast(epochs,model_ast,feature_extractor_ast,device):
    predictions,probabilities=[],[]
    with torch.no_grad():
        for epoch in epochs:
            features=feature_extractor_ast(epoch,sampling_rate=16000,return_tensors="pt")
            outputs=model_ast(features["input_values"].to(device));probs=torch.softmax(outputs.logits,dim=-1)[0].cpu().numpy()
            predictions.append(int(np.argmax(probs)));probabilities.append(probs.tolist())
    return predictions,probabilities
def analyze_ast_results(predictions,probabilities):
    predictions=np.array(predictions);pd_ratio=np.mean(predictions==1)*100
    if pd_ratio>=70:diagnosis,risk="Có dấu hiệu Parkinson rõ rệt","high"
    elif pd_ratio>=40:diagnosis,risk="Có dấu hiệu Parkinson, cần theo dõi thêm","medium"
    elif pd_ratio>=20:diagnosis,risk="Có một số dấu hiệu bất thường","low"
    else:diagnosis,risk="Không có dấu hiệu Parkinson rõ rệt","normal"
    return {"diagnosis":diagnosis,"risk_level":risk,"pd_ratio":round(pd_ratio,2),"total_epochs":len(predictions),"pd_epochs":int(np.sum(predictions==1)),"hc_epochs":int(np.sum(predictions==0))}
def plot_epoch_predictions(probabilities,predictions):
    fig,ax=plt.subplots(figsize=(15,5));epoch_indices=np.arange(len(predictions));pd_probs=[p[1] for p in probabilities]
    colors=['#d1495b' if pred==1 else '#66a182' for pred in predictions]
    ax.bar(epoch_indices,pd_probs,color=colors,alpha=0.8);ax.axhline(y=0.5,color='black',linestyle='--',linewidth=1,label='Ngưỡng quyết định (0.5)')
    ax.set_xlabel('Phân đoạn (Epoch) 2 giây',fontsize=12);ax.set_ylabel('Xác suất là Parkinson',fontsize=12);ax.set_title('Phân tích chi tiết dự đoán trên từng phân đoạn',fontsize=16,weight='bold')
    ax.set_ylim(0,1);ax.grid(axis='y',linestyle='--',alpha=0.6);red_patch=mpatches.Patch(color='#d1495b',label='Dự đoán Parkinson');green_patch=mpatches.Patch(color='#66a182',label='Dự đoán Khỏe mạnh')
    ax.legend(handles=[red_patch,green_patch,ax.get_lines()[0]]);plt.tight_layout();return fig
def generate_cnn_explanation_image(file_path,model_cnn,device):
    # (Giữ nguyên logic hàm, chỉ bỏ status_container)
    raw = mne.io.read_raw_bdf(file_path, preload=True, verbose=False).crop(0, 30); raw.filter(1., 40., fir_design='firwin', verbose=False)
    ica = mne.preprocessing.ICA(n_components=10, random_state=97, max_iter=800, verbose=False); ica.fit(raw)
    try: eog_indices,_=ica.find_bads_eog(raw,ch_name='Fp1');ica.exclude=eog_indices
    except Exception: ica.exclude=[]
    raw_clean=ica.apply(raw.copy(),verbose=False).pick_types(eeg=True);eeg_data_original=raw_clean.get_data();n_chans_raw=eeg_data_original.shape[0];eeg_data_adjusted=eeg_data_original
    if n_chans_raw!=model_cnn.conv1.in_channels:
        if n_chans_raw<model_cnn.conv1.in_channels:padding=np.zeros((model_cnn.conv1.in_channels-n_chans_raw,eeg_data_original.shape[1]));eeg_data_adjusted=np.vstack((eeg_data_original,padding))
        else: eeg_data_adjusted=eeg_data_original[:model_cnn.conv1.in_channels,:]
    x_tensor=torch.tensor(eeg_data_adjusted,dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad(): outputs=model_cnn(x_tensor);_,predicted_class_idx=torch.max(outputs,1)
    target_class=predicted_class_idx.item();ig=IntegratedGradients(model_cnn);attr=ig.attribute(x_tensor,target=target_class,n_steps=50).detach().cpu().numpy()[0]
    raw_resampled=raw_clean.copy().resample(sfreq=128);x_data_resampled=raw_resampled.get_data()
    if x_data_resampled.shape[1]==0:return None
    factor=eeg_data_original.shape[1]//x_data_resampled.shape[1];attr_ds=attr[:n_chans_raw,::factor];attr_norm=np.abs(attr_ds)**0.5
    if attr_norm.max()>0:attr_norm/=attr_norm.max()
    channels_of_interest=['C3','Cz','C4','F3','F4','P3','P4'];fig,axes=plt.subplots(len(channels_of_interest),1,figsize=(16,2.5*len(channels_of_interest)),sharex=True,squeeze=False)
    axes=axes.flatten()
    for i,ch_name in enumerate(channels_of_interest):
        ax=axes[i]
        if ch_name in raw_resampled.ch_names:
            ch_idx=raw_resampled.ch_names.index(ch_name);y,a=x_data_resampled[ch_idx],attr_norm[ch_idx]
            points=np.array([np.arange(len(y)),y]).T.reshape(-1,1,2);segments=np.concatenate([points[:-1],points[1:]],axis=1)
            lc=LineCollection(segments,cmap='inferno',norm=plt.Normalize(0,1));lc.set_array(a);lc.set_linewidth(2)
            ax.add_collection(lc);ax.autoscale_view();ax.set_ylabel(ch_name,fontsize=12);ax.grid(True,linestyle='--',alpha=0.4)
    axes[-1].set_xlabel("Thời gian (Mẫu)",fontsize=12);fig.suptitle(f"Giải thích của AI (dựa trên EEGCNN1D)\\n(Lý do mô hình nghiêng về dự đoán Lớp {target_class})",fontsize=18)
    sm=plt.cm.ScalarMappable(cmap=plt.cm.inferno,norm=plt.Normalize(vmin=0,vmax=1));cbar=fig.colorbar(sm,ax=axes.tolist(),orientation='vertical',fraction=0.02,pad=0.01)
    cbar.set_label('Mức Độ Quan Trọng',fontsize=12);plt.tight_layout(rect=[0,0,0.98,0.94]);return fig
# ==================== HÀM MỚI: XỬ LÝ CHO TRỰC QUAN HÓA ====================
def load_eeg_bdf_for_viz(file_path):
    raw=mne.io.read_raw_bdf(file_path,preload=True,verbose=False);raw.pick_types(eeg=True,verbose=False)
    raw.filter(1.,40.,fir_design='firwin',verbose=False);raw.resample(128,verbose=False);data=raw.get_data()
    return data,raw.info['sfreq'],raw.ch_names
def compute_band_powers(eeg_data,sfreq):
    freqs=np.fft.rfftfreq(eeg_data.shape[1],1/sfreq);psd=np.abs(np.fft.rfft(eeg_data,axis=1))**2
    bands={"Delta (0.5–4 Hz)":(0.5,4),"Theta (4–8 Hz)":(4,8),"Alpha (8–13 Hz)":(8,13),"Beta (13–30 Hz)":(13,30),"Gamma (30–40 Hz)":(30,40)}
    band_powers={}
    for band_name,(low,high) in bands.items():
        idx=np.logical_and(freqs>=low,freqs<=high);band_powers[band_name]=np.mean(psd[:,idx],axis=1)
    return band_powers,freqs,psd
# ==================== GIAO DIỆN NGƯỜI DÙNG (UI) ====================
st.title("🧠 Ứng dụng Chẩn đoán Bệnh Parkinson từ tín hiệu EEG")
st.markdown("Tải lên một file `.bdf` để mô hình AI phân tích và đưa ra chẩn đoán cũng như giải thích.")

model_ast, feature_extractor_ast, model_cnn, device = load_models()
uploaded_file = st.file_uploader("Chọn một file .bdf", type=["bdf"])

# Khởi tạo session_state để đánh dấu file đã được xử lý hay chưa
if 'processed_file_name' not in st.session_state:
    st.session_state['processed_file_name'] = None

if uploaded_file is not None:
    # CHỈ CHẠY PHÂN TÍCH KHI TẢI LÊN FILE MỚI
    if st.session_state.processed_file_name != uploaded_file.name:
        st.info(f"Đã tải lên file mới: `{uploaded_file.name}`. Bắt đầu xử lý...")
        with st.spinner("Đang thực hiện phân tích toàn diện, vui lòng chờ..."):
            try:
                # Đọc nội dung file một lần
                file_content = uploaded_file.getvalue()

                # Gọi hàm xử lý nặng và cache kết quả
                (analysis_result, epoch_chart_fig, explanation_fig,
                 eeg_data_viz, sfreq_viz, ch_names_viz,
                 band_powers_viz, freqs_viz, psd_viz) = run_full_analysis(file_content)

                # Lưu tất cả kết quả vào session_state
                st.session_state['analysis_result'] = analysis_result
                st.session_state['epoch_chart_fig'] = epoch_chart_fig
                st.session_state['explanation_fig'] = explanation_fig
                st.session_state['eeg_data_viz'] = eeg_data_viz
                st.session_state['sfreq_viz'] = sfreq_viz
                st.session_state['ch_names_viz'] = ch_names_viz
                st.session_state['band_powers_viz'] = band_powers_viz
                st.session_state['freqs_viz'] = freqs_viz
                st.session_state['psd_viz'] = psd_viz

                # Đánh dấu là đã xử lý xong file này
                st.session_state.processed_file_name = uploaded_file.name
                st.success("Phân tích hoàn tất!")

            except Exception as e:
                st.error(f"Đã xảy ra lỗi trong quá trình phân tích: {e}")
                import traceback
                st.error(traceback.format_exc())
                # Reset trạng thái nếu có lỗi
                st.session_state.processed_file_name = None

    # --- PHẦN HIỂN THỊ LUÔN CHẠY, LẤY DỮ LIỆU TỪ SESSION_STATE ---
    if st.session_state.processed_file_name is not None:
        # Lấy kết quả từ session_state
        analysis_result = st.session_state['analysis_result']
        epoch_chart_fig = st.session_state['epoch_chart_fig']
        explanation_fig = st.session_state['explanation_fig']

        st.subheader("✅ Kết quả chẩn đoán (từ mô hình AST)")
        risk_level = analysis_result['risk_level']
        color_map = {"high": "red", "medium": "orange", "low": "gold", "normal": "green"}
        risk_color = color_map.get(risk_level, "black")
        st.markdown(f"**Chẩn đoán:** {analysis_result['diagnosis']}")
        st.markdown(f"**Mức độ rủi ro:** <span style='color: {risk_color}; font-weight: bold;'>{risk_level.upper()}</span>", unsafe_allow_html=True)
        st.metric(label="Tỷ lệ phân đoạn Parkinson", value=f"{analysis_result['pd_ratio']}%")
        st.text(f"Chi tiết: {analysis_result['pd_epochs']} phân đoạn Parkinson / {analysis_result['hc_epochs']} phân đoạn khỏe mạnh.")
        st.divider()

        if epoch_chart_fig:
            st.subheader("📊 Biểu đồ phân tích chi tiết")
            st.markdown("Biểu đồ dưới đây thể hiện xác suất dự đoán là 'Parkinson' cho mỗi phân đoạn (epoch) 2 giây. Màu đỏ cho thấy dự đoán là Parkinson, màu xanh là Khỏe mạnh.")
            st.pyplot(epoch_chart_fig)
            st.divider()

        if explanation_fig:
            st.subheader("🧠 Hình ảnh Giải thích (từ mô hình EEGCNN1D)")
            st.pyplot(explanation_fig)
            st.divider()

        # PHẦN TRỰC QUAN HÓA TƯƠNG TÁC
        with st.expander("🔬 Khám phá và Trực quan hóa Tín hiệu EEG Tương tác"):
            # Lấy dữ liệu trực quan hóa từ session_state
            band_powers_viz = st.session_state['band_powers_viz']
            ch_names_viz = st.session_state['ch_names_viz']
            eeg_data_viz = st.session_state['eeg_data_viz']
            freqs_viz = st.session_state['freqs_viz']
            psd_viz = st.session_state['psd_viz']

            st.markdown("### 📊 Trung bình công suất theo dải tần")
            avg_powers = {band: np.mean(val) for band, val in band_powers_viz.items()}
            st.bar_chart(avg_powers)

            st.markdown("### 🎛️ Tùy chọn hiển thị")
            col_ch, col_band = st.columns(2)
            with col_ch:
                selected_channel = st.selectbox("Chọn kênh EEG:", ch_names_viz, index=0, key="viz_channel")
                ch_idx = ch_names_viz.index(selected_channel)
            with col_band:
                selected_band = st.selectbox("Chọn dải tần hiển thị:", list(band_powers_viz.keys()), index=2, key="viz_band")

            st.markdown(f"### 🧩 Tín hiệu EEG – Kênh **{selected_channel}**")
            st.line_chart(eeg_data_viz[ch_idx])

            st.markdown("### 🔬 Phổ tần (Power Spectral Density)")
            fig_psd, ax_psd = plt.subplots(figsize=(10, 4))
            ax_psd.plot(freqs_viz, psd_viz[ch_idx], color="#ff6f3c", lw=1.5)
            ax_psd.set_title(f"Phổ tần của kênh {selected_channel}")
            ax_psd.set_xlabel("Tần số (Hz)")
            ax_psd.set_ylabel("Công suất (μV²/Hz)")

            bands_def = { "Delta (0.5–4 Hz)": (0.5, 4), "Theta (4–8 Hz)": (4, 8), "Alpha (8–13 Hz)": (8, 13), "Beta (13–30 Hz)": (13, 30), "Gamma (30–40 Hz)": (30, 40) }
            low_lim, high_lim = bands_def[selected_band]
            ax_psd.axvspan(low_lim, high_lim, alpha=0.25, color="#ff6f3c", label=selected_band)
            ax_psd.legend()
            ax_psd.grid(True, linestyle='--', alpha=0.6)
            st.pyplot(fig_psd)
else:
    # Nếu không có file nào được tải lên, reset trạng thái
    st.session_state.processed_file_name = None
    st.info("Chờ bạn tải file lên...")

Overwriting app.py


In [ ]:
from getpass import getpass

# Yêu cầu người dùng nhập authtoken
authtoken = getpass("Dán authtoken ngrok của bạn vào đây rồi nhấn Enter: ")

# Cấu hình ngrok với authtoken đã nhập
!ngrok config add-authtoken {authtoken}

Dán authtoken ngrok của bạn vào đây rồi nhấn Enter: ··········
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!kill $(ps aux | grep streamlit | awk '{print $2}')


zsh:kill:1: kill 1374 failed: no such process
zsh:kill:1: kill 1372 failed: no such process


In [4]:
import time
from pyngrok import ngrok
import os

# --- CHẠY ỨNG DỤNG STREAMLIT TRONG NỀN ---
# Dấu '&' ở cuối sẽ chạy lệnh này trong background
# 'nohup' đảm bảo tiến trình không bị ngắt
# '> streamlit.log 2>&1' sẽ ghi log vào file thay vì in ra màn hình
!nohup streamlit run app.py > streamlit.log 2>&1 &

# Đợi khoảng 5 giây để server Streamlit có thời gian khởi động
print("Đang chờ server Streamlit khởi động...")
time.sleep(5)

# --- TẠO TUNNEL VỚI NGROK ---
# Bây giờ chúng ta mới tạo tunnel sau khi server đã chắc chắn chạy
try:
    public_url = ngrok.connect(8501)
    print("🎉🎉🎉 Ứng dụng của bạn đã sẵn sàng! 🎉🎉🎉")
    print(f"Truy cập vào đường link công khai sau: {public_url}")
except Exception as e:
    print(f"Lỗi khi kết nối ngrok: {e}")
    print("\nHãy thử kiểm tra lại các bước sau:")
    print("1. Đã chạy ô cấu hình authtoken của ngrok chưa?")
    print("2. Authtoken bạn dán vào có chính xác không?")
    print("3. Thử vào menu 'Runtime' -> 'Restart session' rồi chạy lại từ đầu.")

ModuleNotFoundError: No module named 'pyngrok'